The goal of this project is :
 - Identify main reason for customer satisfaction or dissatisfaction
 - Predict satisfaction level by category and product type


In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('rslp')
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
#import string
%matplotlib inline


import numpy
import matplotlib.pyplot as plt
import string
import re
from gensim import corpora
from gensim.models import Phrases
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Error loading wordnet: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>
[nltk_data] Error loading stopwords: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>
[nltk_data] Error loading rslp: <urlopen error [Errno 8] nodename nor
[nltk_data]     servname provided, or not known>


ModuleNotFoundError: No module named 'gensim'

# Preprocessing

In [ ]:
reviews = pd.read_csv('brazilian-ecommerce/olist_order_reviews_dataset.csv', header=1 )
reviews
reviews.review_comment_title = reviews.review_comment_title.fillna('')
reviews.review_comment_message = reviews.review_comment_message.fillna('')
reviews["review_comment"] = reviews['review_comment_title'].map(str) + ' ' + reviews['review_comment_message'].map(str)
reviews['review_comment'] = reviews['review_comment'].apply(lambda x: ' '.join(pd.unique(x.split())))
#reviews_only = reviews[pd.notnull(reviews['review_comment'])]
reviews.head()
reviews = reviews[pd.notnull(reviews['review_comment_title']) | pd.notnull(reviews['review_comment_message'])]
reviews['review_comment']  = reviews['review_comment'] .str.replace('[^\w\s]','')
reviews['review_comment']  = reviews['review_comment'].str.lower()
reviews.head()



In [ ]:
from nltk.stem.porter import PorterStemmer
stemmer = nltk.stem.RSLPStemmer()
stopwords = nltk.corpus.stopwords.words('portuguese')
#stopwords = set(stopwords) - set(('não'))
#noise_words = ['recebemo', 'recebimento', 'nan','produto','correio', 'entregu','recomendo','compra', 'comprar','recebi', 'compra', 'entregu', 'entrega', 'entregoa', 'comprei',  'veio', 'é', 'pedido' , 'chegou', 'apena']
#recebemo" + 0.009*"funciona" + 0.009*"recebimento"
#" + 0.025*"veio" + 0.024*"é" + 0.017*"entregu" + 0.013*"pedido" + 0.013*"chegou" + 0.011*"apena" + 0.011*"qualidad" + 0.010*"recomendo"



reviews['review_comment_t']  = reviews['review_comment'] .str.replace('[^\w\s]','')
reviews['review_comment_t']  = reviews['review_comment_t'] .str.lower().str.split()
#reviews['review_comment_t'] =  reviews['review_comment_t'].apply(lambda x: [item for item in x if item not in stopwords ])

#reviews['review_comment_t'] =  reviews['review_comment_t'].apply(lambda x: [item for item in x if item not in noise_words ])

reviews['review_comment_t_stem'] = reviews['review_comment_t'].apply(lambda x: [stemmer.stem(y) for y in x])
reviews[reviews['review_score']<3][['review_comment','review_comment_t']].head()
reviews

# Exploratory sentiment analysis

In [ ]:
reviews['review_pos_neg'] = reviews['review_score'] 
reviews.head()

#reviews_only['review_pos_neg'] ([0, 1, 2, 3, 4, 5], ['neg', 'neg', 'neg', 'neg', 'pos', 'pos'])
reviews['review_pos_neg'] = reviews['review_pos_neg'].map({1: 'neg', 2: 'neg', 3: 'neg', 4: 'neg', 5:'pos' })
reviews.head()

In [ ]:

df = reviews[['review_comment_t_stem', 'review_pos_neg']]
df.apply(tuple, axis=1)
df.head()

In [ ]:
sum = []
for wlist in reviews['review_comment_t_stem']:
  sum = sum + wlist

In [ ]:
mergedlist = list(sum)
documents = [tuple(x) for x in df.values]

In [ ]:
all_words = nltk.FreqDist(mergedlist)
word_features = list(all_words)[:600]
word_features

In [ ]:
def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features
word_features

In [ ]:
featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print('Accuracy: ', nltk.classify.accuracy(classifier, test_set))
print(classifier)
print(classifier.show_most_informative_features(400))
print(classifier.most_informative_features(400))



Using exploratory sentiment analysis revieled few main reson for satisfaction/dissatisfaction: 
- Quality of a product(s) - senatnces describing color, size, diemntions, aesthetic qualities inducate that review is about quality, it could be both positive and negative.  
- Quantity of product(s) - lack of full order. Mostly in negative tone, this reviews are ususlly complaints about the difference between the number of products aht was ordered and teh number of products that was delivered.
- Delivery - There are a lot of verbs describing delivery , verb  'delivered' (entregue) and others may not signify any connotation and may appear in othe contexts. And there adverbes that refer to timing that actually means that the comment is about the delivery.
- Costumer service - there are some verbs that may refer to costmer servis: email, phone.

While this sentiment analysis may show how strong a word is associated with the reason for satisfaction or dissaisfaction with an order.




In order to process review data was labeled, manual lableing revieled more resons, some of them more common: product was not delivered at all, and some of them are less common, i.e. wrong item. Costumer service category proved to be an uncommun complaint too.

In [ ]:
quality_i = ["defeito", 'menor','branca','quebrado', 'diferente', 'foto', "descrição",'falso', 'falsificado','condiz', 'riscos', 'proteção','danificado']
delivery_i = ["rapido", 'antes','depois', 'prazo de entrega', 'riscos', 'proteção','danificado']



In [ ]:
reviews = pd.read_csv('brazilian-ecommerce/olist_order_reviews_dataset.csv', header=1 )
reviews
reviews.review_comment_title = reviews.review_comment_title.fillna('')
reviews.review_comment_message = reviews.review_comment_message.fillna('')
reviews["review_comment"] = reviews['review_comment_title'].map(str) + ' ' + reviews['review_comment_message'].map(str)
reviews['review_comment'] = reviews['review_comment'].apply(lambda x: ' '.join(pd.unique(x.split())))
reviews.head()
reviews = reviews[pd.notnull(reviews['review_comment_title']) | pd.notnull(reviews['review_comment_message'])]
reviews['review_comment']  = reviews['review_comment'] .str.replace('[^\w\s]','')
reviews['review_comment']  = reviews['review_comment'].str.lower()
reviews.head()


In [ ]:
list(reviews.columns.values)

In [ ]:
# dimention reduction algorithm may pca ica

randomly selected fields were labeled :

In [ ]:
reviews_480 = pd.read_csv('brazilian-ecommerce/olist_order_reviews_dataset_manual_review_480__.csv', encoding="cp1252")

In [ ]:
reviews_480.review_comment_title = reviews_480.review_comment_title.fillna('')
reviews_480.review_comment_message = reviews_480.review_comment_message.fillna('')
reviews_480["review_comment"] = reviews_480['review_comment_title'].map(str) + ' ' + reviews_480['review_comment_message'].map(str)
reviews_480['review_comment'] = reviews_480['review_comment'].apply(lambda x: ' '.join(pd.unique(x.split())))
#reviews_only = reviews[pd.notnull(reviews['review_comment'])]
reviews_480.head()
reviews_480 = reviews_480[pd.notnull(reviews_480['review_comment_title']) | pd.notnull(reviews_480['review_comment_message'])]
reviews_480['review_comment']  = reviews_480['review_comment'] .str.replace('[^\w\s]','')
reviews_480['review_comment']  = reviews_480['review_comment'].str.lower()
reviews_480.head()


In [ ]:
list(reviews_480.columns.values)

In [ ]:
reviews_480
reviews_480[['delivery_pos',
 'quality_pos',
 'quantity_pos',
 'cs_pos',
 'price_pos',
 'delivery',
 'delivery_not_del',
 'quality',
 'quantity_parcial',
 'wrong_item',
 'cs']] = reviews_480[['delivery_pos',
 'quality_pos',
 'quantity_pos',
 'cs_pos',
 'price_pos',
 'delivery',
 'delivery_not_del',
 'quality',
 'quantity_parcial',
 'wrong_item',
 'cs']].fillna(value=0)

In [ ]:
len(reviews_480)

In [ ]:
reviews_480

#Sentiment Analysis

In [ ]:
reviewsm = pd.read_csv('brazilian-ecommerce/olist_order_reviews_dataset_manual_review_.csv')
reviews = pd.read_csv('brazilian-ecommerce/olist_order_reviews_dataset.csv')
#reviewsm = reviewsm [['review_id',	'delivery_pos',	'quality_pos', 	'quantity_pos',	'cs_pos'	,'price_pos']]


reviewsm.review_comment_title = reviewsm.review_comment_title.fillna('')
reviewsm.review_comment_message = reviewsm.review_comment_message.fillna('')

reviewsm["review_comment"] = reviewsm['review_comment_title'].map(str) + ' ' + reviewsm['review_comment_message'].map(str)
reviewsm['review_comment'] = reviewsm['review_comment'].apply(lambda x: ' '.join(pd.unique(x.split())))

#reviews_only = reviews[pd.notnull(reviews['review_comment'])]

reviewsm = reviewsm[pd.notnull(reviewsm['review_comment_title']) | pd.notnull(reviewsm['review_comment_message'])]

reviewsm['review_comment']  = reviewsm['review_comment'] .str.replace('[^\w\s]','')
reviewsm['review_comment']  = reviewsm['review_comment'] .str.lower()



reviewsm_pos = reviewsm[pd.notnull(reviewsm['delivery_pos']) & pd.notnull(reviewsm['quality_pos']) & pd.notnull(reviewsm['quantity_pos']) &  pd.notnull(reviewsm['cs_pos'])& pd.notnull(reviewsm['price_pos'])
                   #|pd.notnull(reviewsm['delivery']) | pd.notnull(reviewsm['delivery_not_del'])| pd.notnull(reviewsm['quality'])| 
               # pd.notnull(reviewsm['quantity/parcial'])| pd.notnull(reviewsm['wrong_item'])| pd.notnull(reviewsm['cs'])
                   ]


reviewsm_pos_n = reviewsm[(pd.notnull(reviewsm['delivery_pos']) & pd.notnull(reviewsm['quality_pos']) & pd.notnull(reviewsm['quantity_pos']) &  pd.notnull(reviewsm['cs_pos'])& pd.notnull(reviewsm['price_pos']))
                 |  (pd.notnull(reviewsm['delivery']) & pd.notnull(reviewsm['delivery_not_del'])& pd.notnull(reviewsm['quality'])& 
               pd.notnull(reviewsm['quantity_parcial'])& pd.notnull(reviewsm['wrong_item'])& pd.notnull(reviewsm['cs'])
                   )]




#len(reviewsm.dropna())
#reviewsm.head()
#reviewsm = reviewsm.dropna()
#reviewsm_ = pd.merge(reviews, reviews, left_on='review_id')
  

In [ ]:
len(reviewsm_pos_n)

In [ ]:
reviewsm_pos_n_c = reviewsm_pos_n.append(reviews_480) 

In [ ]:
reviewsm_pos_n_c
len(reviewsm_pos_n_c)

In [ ]:
reviewsm_pos_n=reviewsm_pos_n_c

In [ ]:
reviewsm_pos_n

In [ ]:
reviewsm_pos_n.loc[reviewsm_pos_n.delivery_pos >= 1, 'delivery_categories'] = 'positive' 
reviewsm_pos_n.loc[reviewsm_pos_n.delivery >=1, 'delivery_categories'] = 'negative' 
reviewsm_pos_n.loc[reviewsm_pos_n.delivery_not_del >=1, 'delivery_categories'] = 'negative' 


reviewsm_pos_n.loc[reviewsm_pos_n.quality_pos >= 1, 'quality_categories'] = 'positive' 
reviewsm_pos_n.loc[reviewsm_pos_n.quality >=1, 'quality_categories'] = 'negative' 



reviewsm_pos_n.loc[reviewsm_pos_n.quantity_pos >= 1, 'quantity_categories'] = 'positive' 
reviewsm_pos_n.loc[reviewsm_pos_n.quantity_parcial >=1, 'quantity_categories'] = 'negative' 

reviewsm_pos_n.loc[reviewsm_pos_n.wrong_item >=1, 'quantity_categories'] = 'negative' 



reviewsm_pos_n.loc[reviewsm_pos_n.wrong_item >= 1, 'wrong_item_categories'] = 'negative' 

reviewsm_pos_n.loc[reviewsm_pos_n.quantity_pos >=1, 'quant_wrong_categories'] = 'negative' 



reviewsm_pos_n.loc[reviewsm_pos_n.wrong_item >= 1, 'quant_wrong_categories'] = 'negative' 





#reviewsm_pos_n.loc[reviewsm_pos_n.delivery_not_del <1 & reviewsm_pos_n.delivery <1 & reviewsm_pos_n.delivery_pos < 1, 'delivery_categories'] = 'no_mention' 

reviewsm_pos_n['delivery_categories'] = reviewsm_pos_n['delivery_categories'].fillna('no_mention')
reviewsm_pos_n['quality_categories'] = reviewsm_pos_n['quality_categories'].fillna('no_mention')

reviewsm_pos_n['quantity_categories'] = reviewsm_pos_n['quantity_categories'].fillna('no_mention')


reviewsm_pos_n['wrong_item_categories'] = reviewsm_pos_n['wrong_item_categories'].fillna('no_mention')


reviewsm_pos_n['quant_wrong_categories'] = reviewsm_pos_n['quant_wrong_categories'].fillna('no_mention')








reviewsm_pos.head()


reviewsm_pos_n




In [ ]:
len(reviewsm_pos_n.iloc[:729])

In [ ]:
reviewsm_pos_n_f = reviewsm_pos_n.iloc[:729]
reviewsm_pos_n_f

In [ ]:
reviewsm_pos_n = reviewsm_pos_n_f

# Visualization for categories 

In [ ]:

import matplotlib.patches as mpatches


fig = plt.figure()

plt.subplot(2, 4, 1)
s = pd.Series(reviewsm_pos_n.delivery_categories)
s = s.value_counts(normalize=True)
s.plot.bar()
plt.title(r'delivery_categories')



plt.subplot(2, 4, 3)
s = pd.Series(reviewsm_pos_n.wrong_item_categories)
s = s.value_counts(normalize=True)
s.plot.bar()
plt.title(r'wrong_item_categories')

plt.subplot(2, 4, 6)
s = pd.Series(reviewsm_pos_n.quality_categories)
s = s.value_counts(normalize=True)
s.plot.bar()
plt.title(r'quality_categories')

plt.subplot(2, 4, 8)
plt.title(r'quantity_categories')
s = pd.Series(reviewsm_pos_n.quantity_categories)
s = s.value_counts(normalize=True)
s.plot.bar()


plt.show()

In [ ]:
print ('delivery_categories\n',reviewsm_pos_n.delivery_categories.value_counts(dropna=False))

In [ ]:
print ('wrong_item_categories\n',reviewsm_pos_n.wrong_item_categories.value_counts(dropna=False))

In [ ]:
print ('quality_categories\n',reviewsm_pos_n.quality_categories.value_counts(dropna=False))

In [ ]:
print ('quantity_categories\n',reviewsm_pos_n.quantity_categories.value_counts(dropna=False))

In [ ]:
print(reviewsm_pos_n.info())

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 3), stop_words='english')
features = tfidf.fit_transform(reviewsm_pos_n.review_comment).toarray()
labels = reviewsm_pos_n.delivery_categories
features.shape

In [ ]:
reviewsm_chi2= reviewsm_pos_n
list(reviewsm_chi2.columns.values)

In [ ]:
reviewsm_chi2= reviewsm_pos_n
reviewsm_chi2
reviews['review_pos_neg'] = reviews['review_pos_neg'].map({1: 'neg', 2: 'neg', 3: 'neg', 4: 'neg', 5:'pos' })
reviews.head()

In [ ]:
from sklearn.feature_selection import chi2
import numpy as np
N = 2
for delivery_, delivery_categories in sorted(reviewsm_pos_n.items()):
  features_chi2 = chi2(features, labels == delivery_categories)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  trigrams = [v for v in feature_names if len(v.split(' ')) == 3]
  print("# '{}':".format(delivery_))
  print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
  print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))
  print("  . Most correlated trigrams:\n. {}".format('\n. '.join(trigrams[-N:])))

In [ ]:
from sklearn.feature_selection import chi2
import numpy as np
N = 2
for delivery_, quality_categories in sorted(reviewsm_pos_n.items()):
  features_chi2 = chi2(features, labels == delivery_categories)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  trigrams = [v for v in feature_names if len(v.split(' ')) == 3]
  print("# '{}':".format(delivery_))
  print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
  print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))
  print("  . Most correlated trigrams:\n. {}".format('\n. '.join(trigrams[-N:])))

In [ ]:
featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print('Accuracy: ', nltk.classify.accuracy(classifier, test_set))
print(classifier)
print(classifier.show_most_informative_features(400))
print(classifier.most_informative_features(400))

## delivery_categories

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 3), stop_words='english')
features = tfidf.fit_transform(reviewsm_pos_n.review_comment).toarray()
labels = reviewsm_pos_n.delivery_categories
features.shape

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(reviewsm_pos_n['review_comment'], reviewsm_pos_n['delivery_categories'], random_state = 0)
count_vect = CountVectorizer(ngram_range=(1, 3))
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='f1_weighted', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
import seaborn as sns
sns.boxplot(x='model_name', y='accuracy', data=cv_df)
sns.stripplot(x='model_name', y='accuracy', data=cv_df, 
              size=8, jitter=True, edgecolor="gray", linewidth=2)
plt.show()

In [ ]:
print(cv_df.groupby('model_name').accuracy.mean())


In [ ]:

def mod_select(cat):
    tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 3), stop_words='english')
    features = tfidf.fit_transform(reviewsm_pos_n.review_comment).toarray()
    labels = reviewsm_pos_n[cat]
    features.shape
    
   
    models = [
        RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
        LinearSVC(),
        SVC(C = 700, gamma = 0.001, kernel = 'rbf' ),
        MultinomialNB(),
        LogisticRegression(random_state=0),
    ]
    CV = 5
    cv_df = pd.DataFrame(index=range(CV * len(models)))
    entries = []
    for model in models:
      model_name = model.__class__.__name__
      accuracies = cross_val_score(model, features, labels, scoring='f1_weighted', cv=CV)
      for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
    cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
    import seaborn as sns
    sns.boxplot(x='model_name', y='accuracy', data=cv_df)
    sns.stripplot(x='model_name', y='accuracy', data=cv_df, 
                  size=8, jitter=True, edgecolor="gray", linewidth=2)
    plt.show()
    
    print(cv_df.groupby('model_name').accuracy.mean())



In [ ]:
mod_select('delivery_categories')

In [ ]:
reviews

In [ ]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 3), stop_words='english')
features = tfidf.fit_transform(reviewsm_pos_n.review_comment).toarray()
labels = reviewsm_pos_n['delivery_categories']


model  = SVC(C=700, gamma =0.001, degree = 0, kernel='rbf')
model.fit(features, labels)
reviews['predicted_delivery'] = model.predict(tfidf.transform( reviews['review_comment']).toarray())

In [ ]:
reviews['predicted_delivery'] = model.predict(tfidf.transform( reviews['review_comment']).toarray())

In [ ]:
reviews

In [ ]:

from sklearn.model_selection import GridSearchCV


   



# Create the hyperparameter grid
c_space = np.logspace(-5, 8, 15)
param_grid = {'C': c_space, 'penalty': ['l1', 'l2']}


param_grid = [
  {'C': [1, 10, 100, 1000, 1100], 'kernel': ['linear']},
  {'C': [1, 10, 100, 200, 300, 500, 600, 700, 740, 800, 900, 1000, 1100], 'gamma': [0.01, 0.0008, 0.001, 0.002, 0.0001],  'kernel': ['rbf']},
  #{'C': [1, 10, 100, 1000, 1100], 'gamma': [0.01, 0.001, 0.0001], 'degree' : [0, 1, 2, 3, 4, 5, 6], 'kernel': ['poly']},
    
 ]

# Instantiate the logistic regression classifier: logreg
svc = SVC()

# Instantiate the GridSearchCV object: logreg_cv
svc_cv = GridSearchCV(svc, param_grid, cv=5)

# Fit it to the training data
svc_cv.fit(features, labels)

# Print the optimal parameters and best score
print("Tuned svc Parameter: {}".format(svc_cv.best_params_))
print("Tuned svc Accuracy: {}".format(svc_cv.best_score_))




In [ ]:
mod_select('quality_categories')

In [ ]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 3), stop_words='english')
features = tfidf.fit_transform(reviewsm_pos_n.review_comment).toarray()
labels = reviewsm_pos_n['quality_categories']




In [ ]:

from sklearn.model_selection import GridSearchCV


   



# Create the hyperparameter grid
c_space = np.logspace(-5, 8, 15)
param_grid = {'C': c_space, 'penalty': ['l1', 'l2']}


param_grid = [
  {'C': [1, 10, 100, 1000, 1100], 'kernel': ['linear']},
  {'C': [1, 10, 100, 700, 900, 1000, 1100], 'gamma': [0.01,  0.001,  0.0001],  'kernel': ['rbf']},
  #{'C': [1, 10, 100, 1000, 1100], 'gamma': [0.01, 0.001, 0.0001], 'degree' : [0, 1, 2, 3, 4, 5, 6], 'kernel': ['poly']},
    
 ]

# Instantiate the logistic regression classifier: logreg
svc = SVC()

# Instantiate the GridSearchCV object: logreg_cv
svc_cv = GridSearchCV(svc, param_grid, cv=5)

# Fit it to the training data
svc_cv.fit(features, labels)

# Print the optimal parameters and best score
print("Tuned svc Parameter: {}".format(svc_cv.best_params_))
print("Tuned svc Accuracy: {}".format(svc_cv.best_score_))





In [ ]:
model  = SVC(C=700, gamma =0.001, degree = 0, kernel='rbf')
model.fit(features, labels)
reviews['predicted_quality'] = model.predict(tfidf.transform( reviews['review_comment']).toarray())

In [ ]:
reviews

In [ ]:
mod_select('quantity_categories')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 3), stop_words='english')
features = tfidf.fit_transform(reviewsm_pos_n.review_comment).toarray()
labels = reviewsm_pos_n['quantity_categories']
features.shape

In [ ]:

from sklearn.model_selection import GridSearchCV


   



# Create the hyperparameter grid
c_space = np.logspace(-5, 8, 15)
param_grid = {'C': c_space, 'penalty': ['l1', 'l2']}


param_grid = [
  {'C': [1, 10, 100, 1000, 1100], 'kernel': ['linear']},
  {'C': [1, 10, 100, 200, 300, 500, 600, 700, 740, 800, 900, 1000, 1100], 'gamma': [0.01, 0.0008, 0.001, 0.002,  0.003 , 0.0001],  'kernel': ['rbf']},
  #{'C': [1, 10, 100, 1000, 1100], 'gamma': [0.01, 0.001, 0.0001], 'degree' : [0, 1, 2, 3, 4, 5, 6], 'kernel': ['poly']},
    
 ]

# Instantiate the logistic regression classifier: logreg
svc = SVC()

# Instantiate the GridSearchCV object: logreg_cv
svc_cv = GridSearchCV(svc, param_grid, cv=5)

# Fit it to the training data
svc_cv.fit(features, labels)

# Print the optimal parameters and best score
print("Tuned svc Parameter: {}".format(svc_cv.best_params_))
print("Tuned svc Accuracy: {}".format(svc_cv.best_score_))





In [ ]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 3), stop_words='english')
features = tfidf.fit_transform(reviewsm_pos_n.review_comment).toarray()
labels = reviewsm_pos_n['quantity_categories']


model  = SVC(C=700, gamma =0.002, degree = 0, kernel='rbf')
model.fit(features, labels)
reviews['predicted_quantity'] = model.predict(tfidf.transform( reviews['review_comment']).toarray())

In [ ]:
reviews

In [ ]:
from nltk.stem.porter import PorterStemmer
stemmer = nltk.stem.RSLPStemmer()
stopwords = nltk.corpus.stopwords.words('portuguese')
#stopwords = set(stopwords) - set(('não'))
#noise_words = ['recebemo', 'recebimento', 'nan','produto','correio', 'entregu','recomendo','compra', 'comprar','recebi', 'compra', 'entregu', 'entrega', 'entregoa', 'comprei',  'veio', 'é', 'pedido' , 'chegou', 'apena']
#recebemo" + 0.009*"funciona" + 0.009*"recebimento"
#" + 0.025*"veio" + 0.024*"é" + 0.017*"entregu" + 0.013*"pedido" + 0.013*"chegou" + 0.011*"apena" + 0.011*"qualidad" + 0.010*"recomendo"



reviews['review_comment_t']  = reviews['review_comment'] .str.replace('[^\w\s]','')
reviews['review_comment_t']  = reviews['review_comment_t'] .str.lower().str.split()
#reviews['review_comment_t'] =  reviews['review_comment_t'].apply(lambda x: [item for item in x if item not in stopwords ])

#reviews['review_comment_t'] =  reviews['review_comment_t'].apply(lambda x: [item for item in x if item not in noise_words ])

reviews['review_comment_t_stem'] = reviews['review_comment_t'].apply(lambda x: [stemmer.stem(y) for y in x])
reviews[reviews['review_score']<3][['review_comment','review_comment_t']].head()


In [ ]:
type(stopwords)

In [ ]:
import gensim
dictionary = gensim.corpora.Dictionary(processed_docs_pos)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs_pos]
bow_corpus[4310]

In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=4, id2word=dictionary, passes=2, workers=2)

In [ ]:
#df = reviews_only[['review_comment_message', 'review_score']]

#classification
df = reviews[['review_comment_t_stem', 'review_pos_neg']]



In [ ]:

df.apply(tuple, axis=1)

df.head()

In [ ]:
sum = []
for wlist in reviews['review_comment_t_stem']:
  sum = sum + wlist

In [ ]:
sum

In [ ]:
#mergedlist = list(sum)
documents = [tuple(x) for x in df.values]


In [ ]:
from itertools import chain
all_words = nltk.FreqDist(sum)
#word_features = list(all_words)[:600]
print(type(dict(all_words).keys()))
d=dict(all_words).copy().keys()
word_features ={x for x in chain(dict(all_words).copy().keys()) if x}

In [ ]:
#pd.options.display.max_colwidth = 1000
#print(reviews[reviews['review_comment'].str.contains("atrás")]['review_comment'].shape)
#reviews[reviews['review_comment'].str.contains("atrás")]['review_comment'].head(10)
#mixed

In [ ]:
def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

#random.shuffle(documents)


In [ ]:
#from google.colab import files
#word_df = pd.DataFrame(word_features)
#word_df.to_csv('out.csv', index=False, header=False)
#files.download('out.csv') 

In [ ]:
featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
print('Accuracy: ', nltk.classify.accuracy(classifier, test_set))


In [ ]:
print(classifier)
print(classifier.show_most_informative_features(400))
print(classifier.most_informative_features(400))




#Manual Check
Checkig comments that may inducate a reason fora negative or positive review -  quality, quantity, delivery, packaging 

#Feature Engineering 
Feature Construction: The manual construction of new features from raw data

In [ ]:
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date
o['order_purchase_timestamp']= pd.to_datetime(o['order_purchase_timestamp'])
o['order_approved_at']= pd.to_datetime(o['order_approved_at'])
o['order_delivered_carrier_date']= pd.to_datetime(o['order_delivered_carrier_date'])
o['order_delivered_customer_date']= pd.to_datetime(o['order_delivered_customer_date'])
o['order_estimated_delivery_date']= pd.to_datetime(o['order_estimated_delivery_date'])
o['delivery_delta'] = o['order_delivered_customer_date'] - o['order_purchase_timestamp']
o['delivery_delta'] = o['delivery_delta'] /np.timedelta64(1,'h')

o['est_delta'] = o['order_estimated_delivery_date'] - o['order_purchase_timestamp']
o['est_delta'] = o['est_delta'] /np.timedelta64(1,'h')

o['delivery_est_delta'] = o['est_delta'] - o['delivery_delta'] 
o['delivery_est_delta'] = o['delivery_est_delta']
print(o.describe())
o.head(2)

In [ ]:
# imports
%matplotlib inline

import os
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import string
import re
from gensim import corpora
from gensim.models import Phrases
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from ds_voc.text_processing import TextProcessing

#Joining Data

merging orders, product, payment

In [ ]:
reviews = reviews[['review_id',	'order_id','predicted_delivery',	'predicted_quality',	'predicted_quantity']]

In [ ]:
#join oreder items data, oreder payment, oreder review, oreders dataset, product 

# oreders + order items 


reviews_only = reviews.dropna()
reviews_only.head()
result = pd.merge(o, o_i, how='outer', on=['order_id'])


# add product information 
result = pd.merge(result, p, how='outer', on=['product_id'])
#add reviews
result = pd.merge(result, reviews_only, how='outer', on=['order_id'])
#creating dummies var for categories
result = pd.get_dummies(result, prefix='Category_', columns=['product_category_name'])

# add payments 
result = pd.merge(result, payments, how='outer', on=['order_id'])

result = pd.get_dummies(result, prefix='type', columns=['payment_type'])

#result[['order_id', 'customer_id', 'product_id' , 'seller_id', 'order_item_id', 'review_score']]
#result.drop(['order_purchase_timestamp','order_approved_at',	'order_delivered_carrier_date',	'order_delivered_customer_date',	'order_estimated_delivery_date','shipping_limit_date'])
result.head()

#list(result.columns.values) #time, day product_description_lenght

In [ ]:
result['delivery'] = result['predicted_delivery'].map({'positive': 1 , 'negative':0})
result['quality'] = result['predicted_quality'].map({'positive': 1 , 'negative':0})
result['quantity'] = result['predicted_quantity'].map({'positive': 1 , 'negative':0})



result

In [ ]:
list(result.columns.values)

In [ ]:
result = result.groupby('order_id').agg({
                                        'delivery': 'max',	
                                        'quality': 'max',	
                                        'quantity':'max',                          
                                        'price':'sum', 
                                        'product_photos_qty': 'min','product_photos_qty': 'max', 'product_photos_qty': 'sum', 

                                          'delivery_est_delta': 'max',
                                          'est_delta':'max',
                                           'delivery_delta':'max',                             
                                
                                        'freight_value': 'sum',
                                         
                               
    
                                'type_credit_card': 'sum',
                                           'type_debit_card': 'sum',
                                           'type_not_defined': 'sum',
                                           'type_voucher': 'sum',
                             'payment_sequential': 'max',
 'payment_installments':'max',
 'payment_value':'sum',
                                                                        'Category__pet_shop': 'sum',
                                                                        'Category__portateis_casa_forno_e_cafe': 'sum' ,
                                                                        'Category__agro_industria_e_comercio': 'sum' ,
                                                                        'Category__alimentos': 'sum' ,
                                                                        'Category__alimentos_bebidas': 'sum' ,
                                                                        'Category__artes': 'sum' ,
                                                                        'Category__artes_e_artesanato': 'sum' ,
                                                                        'Category__artigos_de_festas': 'sum' ,
                                                                        'Category__artigos_de_natal': 'sum' ,
                                                                        'Category__audio': 'sum' ,
                                                                        'Category__automotivo': 'sum' ,
                                                                        'Category__bebes': 'sum' ,
                                                                        'Category__bebidas': 'sum' ,
                                                                        'Category__beleza_saude': 'sum' ,
                                                                        'Category__brinquedos': 'sum' ,
                                                                        'Category__cama_mesa_banho': 'sum' ,
                                                                        'Category__casa_conforto': 'sum' ,
                                                                        'Category__casa_conforto_2': 'sum' ,
                                                                        'Category__casa_construcao': 'sum' ,
                                                                        'Category__cds_dvds_musicais': 'sum' ,
                                                                        'Category__cine_foto': 'sum' ,
                                                                        'Category__climatizacao': 'sum' ,
                                                                        'Category__consoles_games': 'sum' ,
                                                                        'Category__construcao_ferramentas_construcao': 'sum' ,
                                                                        'Category__construcao_ferramentas_ferramentas': 'sum' ,
                                                                        'Category__construcao_ferramentas_iluminacao': 'sum' ,
                                                                        'Category__construcao_ferramentas_jardim': 'sum' ,
                                                                        'Category__construcao_ferramentas_seguranca': 'sum' ,
                                                                        'Category__cool_stuff': 'sum' ,
                                                                        'Category__dvds_blu_ray': 'sum' ,
                                                                        'Category__eletrodomesticos': 'sum' ,
                                                                        'Category__eletrodomesticos_2': 'sum' ,
                                                                        'Category__eletronicos': 'sum' ,
                                                                        'Category__eletroportateis': 'sum' ,
                                                                        'Category__esporte_lazer': 'sum' ,
                                                                        'Category__fashion_bolsas_e_acessorios': 'sum' ,
                                                                        'Category__fashion_calcados': 'sum' ,
                                                                        'Category__fashion_esporte': 'sum' ,
                                                                        'Category__fashion_roupa_feminina': 'sum' ,
                                                                        'Category__fashion_roupa_infanto_juvenil': 'sum' ,
                                                                        'Category__fashion_roupa_masculina': 'sum' ,
                                                                        'Category__fashion_underwear_e_moda_praia': 'sum' ,
                                                                        'Category__ferramentas_jardim': 'sum' ,
                                                                        'Category__flores': 'sum' ,
                                                                        'Category__fraldas_higiene': 'sum' ,
                                                                        'Category__industria_comercio_e_negocios': 'sum' ,
                                                                        'Category__informatica_acessorios': 'sum' ,
                                                                        'Category__instrumentos_musicais': 'sum' ,
                                                                        'Category__la_cuisine': 'sum' ,
                                                                        'Category__livros_importados': 'sum' ,
                                                                        'Category__livros_interesse_geral': 'sum' ,
                                                                        'Category__livros_tecnicos': 'sum' ,
                                                                        'Category__malas_acessorios': 'sum' ,
                                                                        'Category__market_place': 'sum' ,
                                                                        'Category__moveis_colchao_e_estofado': 'sum' ,
                                                                        'Category__moveis_cozinha_area_de_servico_jantar_e_jardim': 'sum' ,
                                                                        'Category__moveis_decoracao': 'sum' ,
                                                                        'Category__moveis_escritorio': 'sum' ,
                                                                        'Category__moveis_quarto': 'sum' ,
                                                                        'Category__moveis_sala': 'sum' ,
                                                                        'Category__musica': 'sum' ,
                                                                        'Category__papelaria': 'sum' ,
                                                                        'Category__pc_gamer': 'sum' ,
                                                                        'Category__pcs': 'sum' ,
                                                                        'Category__perfumaria': 'sum' ,
                                                                        'Category__pet_shop': 'sum' ,
                                                                        'Category__portateis_casa_forno_e_cafe': 'sum' ,
                                                                        'Category__portateis_cozinha_e_preparadores_de_alimentos': 'sum' ,
                                                                        'Category__relogios_presentes': 'sum' ,
                                                                        'Category__seguros_e_servicos': 'sum' ,
                                                                        'Category__sinalizacao_e_seguranca': 'sum' ,
                                                                        'Category__tablets_impressao_imagem': 'sum' ,
                                                                        'Category__telefonia': 'sum' ,
                                                                        'Category__telefonia_fixa': 'sum' ,
                                                                        'Category__utilidades_domesticas': 'sum' })

In [ ]:
result_d

In [ ]:
result_d= result.drop(['quality',
 'quantity'], axis=1)
result_d =result_d.dropna()
delivery = result_d['delivery']
result_d= result_d.drop(['delivery'], axis=1)

In [ ]:
result_d, delivery

In [ ]:
models = [
        RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
        LinearSVC(),
        SVC(C = 700, gamma = 0.001, kernel = 'rbf' ),
        MultinomialNB(),
        LogisticRegression(random_state=0),
    ]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, result_d, delivery, scoring='f1_weighted', cv=CV)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
import seaborn as sns
sns.boxplot(x='model_name', y='accuracy', data=cv_df)
sns.stripplot(x='model_name', y='accuracy', data=cv_df, 
                  size=8, jitter=True, edgecolor="gray", linewidth=2)
plt.show()
    
print(cv_df.groupby('model_name').accuracy.mean())



In [ ]:
model_list = [LogisticRegression(), LinearSVC(), RandomForestClassifier() , KNeighborsClassifier(), GaussianNB(), Perceptron() , SGDClassifier(), DecisionTreeClassifier()]
for i in model_list:
  model = i.fit(result_d, delivery)
  print( type(i).__name__, 'Accuracy: ',  np.average(cross_val_score(i, result_d, delivery, scoring='f1_weighted', cv=5)))

In [ ]:

from sklearn.model_selection import GridSearchCV


   



# Create the hyperparameter grid
c_space = np.logspace(-5, 8, 15)
param_grid = {'C': c_space, 'penalty': ['l1', 'l2']}



# Instantiate the logistic regression classifier: logreg
logr = LogisticRegression()

# Instantiate the GridSearchCV object: logreg_cv
logr_cv = GridSearchCV(logr, param_grid, cv=5)

# Fit it to the training data
logr_cv.fit(result_d, delivery)

# Print the optimal parameters and best score
print("Tuned logreg Parameter: {}".format(logr_cv.best_params_))
print("Tuned logreg Accuracy: {}".format(logr_cv.best_score_))





In [ ]:
logr_cv.best_estimator_


In [ ]:
logistic = logr_cv.best_estimator_

In [ ]:
logistic = logr_cv.best_estimator_
coefficients = pd.concat([pd.DataFrame(result_d.columns),pd.DataFrame(np.transpose(logistic.coef_), columns=['coef'])], axis = 1)
coefficients.sort_values(by=['coef'])

In [ ]:
coefficients.to_csv('hey.csv')

In [ ]:
result_q= result.drop(['delivery',
 'quantity'], axis=1)
result_q =result_q.dropna()
quality = result_q['quality']
result_q= result_q.drop(['quality'], axis=1)

model_list = [LogisticRegression(), LinearSVC(), RandomForestClassifier() , KNeighborsClassifier(), GaussianNB(), Perceptron() , SGDClassifier(), DecisionTreeClassifier()]
for i in model_list:
  model = i.fit(result_q, quality)
  print( type(i).__name__, 'Accuracy: ', np.average(cross_val_score(i, result_q, quality, scoring='f1_weighted', cv=5)))

In [ ]:
# Import necessary modules
from scipy.stats import randint
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import  RandomizedSearchCV 


# Setup the parameters and distributions to sample from: param_dist
param_dist = {"max_depth": [3, None],
              "max_features": randint(1, 9),
              "min_samples_leaf": randint(1, 9),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# Instantiate a Decision Tree classifier: tree
tree = RandomForestClassifier()

# Instantiate the RandomizedSearchCV object: tree_cv
tree_cv = RandomizedSearchCV(tree, param_dist, cv=5)

# Fit it to the data
tree_cv.fit(X, y)

# Print the tuned parameters and score
print("Tuned Decision Tree Parameters: {}".format(tree_cv.best_params_))
print("Best score is {}".format(tree_cv.best_score_))


In [ ]:
tree_cv.best_estimator_

In [ ]:
randfor = tree_cv.best_estimator_
coefficients_qual = pd.concat([pd.DataFrame(result_d.columns),pd.DataFrame(np.transpose(randfor.feature_importances_), columns=['feature_importances'])], axis = 1)
coefficients_qual.sort_values(by=['feature_importances'])

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier


In [ ]:
result.head(2)
result['order_id'].duplicated()

In [ ]:
#join oreder items data, oreder payment, oreder review, oreders dataset, product 

# oreders + order items 
result = pd.merge(o, o_i, how='outer', on=['order_id'])


# add product information 
result = pd.merge(result, p, how='outer', on=['product_id'])
#add reviews
result = pd.merge(result, reviews_only, how='outer', on=['order_id'])
#creating dummies var for categories
result = pd.get_dummies(result, prefix='Category_', columns=['product_category_name'])
# add payments 
result = pd.merge(result, payments, how='outer', on=['order_id'])


#result[['order_id', 'customer_id', 'product_id' , 'seller_id', 'order_item_id', 'review_score']]
result.drop(columns=['order_purchase_timestamp','order_approved_at',	'order_delivered_carrier_date',	'order_delivered_customer_date',	'order_estimated_delivery_date','shipping_limit_date'])
list(result.columns.values)


In [ ]:
p

In [ ]:
#payments.order_id.loc['b81ef226f3fe1789b1e8b2acac839d17']
result.loc[payments['order_id'] == '31bc09fdbd701a7a4f9b55b5955b8687']
#payments

In [ ]:
payments['order_id'].duplicated()

In [ ]:

print(result.order_id.nunique())
print(result.customer_id.nunique())

In [ ]:
print(o_i.order_id.nunique())
print(o_i.customer_id.nunique())

In [ ]:
s = pd.read_csv('olist_sellers_dataset.csv')
print(s.info())
print(s.head())

#Reading Files

In [ ]:
payments = pd.read_csv('brazilian-ecommerce/olist_order_payments_dataset.csv')
print(payments.describe())
print(payments.info())
print(payments.head())

In [ ]:
payments.isnull().sum()

In [ ]:
o = pd.read_csv('brazilian-ecommerce/olist_orders_dataset.csv')
print(o.info())
print(o.head())

In [ ]:
o_i = pd.read_csv('brazilian-ecommerce/olist_order_items_dataset.csv')
print(o_i.info())
print(o_i.head())

In [ ]:
category = pd.read_csv('brazilian-ecommerce/product_category_name_translation.csv')
print(category.info())
print(category.head())

In [ ]:

customers = pd.read_csv('brazilian-ecommerce/olist_customers_dataset.csv')
print(customers.info())
print(customers.head())

In [ ]:
geo = pd.read_csv('brazilian-ecommerce/olist_geolocation_dataset.csv')
print(geo.info())
print(geo.head())

In [ ]:
geo_s = pd.read_csv('brazilian-ecommerce/olist_sellers_dataset.csv')
print(geo_s.info())
print(geo_s.head())

In [ ]:
p = pd.read_csv('brazilian-ecommerce/olist_products_dataset.csv')
print(p.info())
print(p.head())